<a href="https://colab.research.google.com/github/diem-ai/topic-modeling/blob/master/model_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### This note book will clean data and create bag of word and tf-idf sparse matrix and save them with help of Pickle. They will be used again when we build LDA, LSA and NMF model.

#### Google Colab Setup

In [30]:
from google.colab import drive
# This will prompt for authorization.
# authorization code: 4/OwErfUj6QceGXhIGx_RWv0MKclb9rilw8UsJnZqFbSez-QS8zQ399JU
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
#import accessory_functions.py from Colab
#https://drive.google.com/open?id=1S7URZIBq4zMh5QWv0qXPHv4ixhgHWN_y

my_module = drive.CreateFile({'id':'1S7URZIBq4zMh5QWv0qXPHv4ixhgHWN_y'})
my_module.GetContentFile('accessory_functions.py')

####  Import libraries

In [33]:
import numpy as np
import string
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')

import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora


import pickle
from wordcloud import WordCloud
# Plotting tools

import matplotlib.pyplot as plt
%matplotlib inline
# Make all my plots 538 Style
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore')

from accessory_functions import preprocess_series_text
from accessory_functions import write_pickle_file


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


<p> Data Path </p>

In [0]:
datapath = '/content/drive/My Drive/data/'


In [0]:
data = pd.read_csv(datapath + 'breakingnews.csv')

data.head(2)

In [0]:
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

#####  cleaning data and save it

In [0]:
processed_docs = preprocess_series_text(documents['headline_text'])



In [0]:
plt.figure(figsize=(15, 15))
wordcloud = WordCloud(
        background_color='white'
#        max_words=200,
        ,max_font_size=20
        , scale=2) # chosen at random by flipping a coin; it was heads)

wordcloud.generate(str(processed_docs))

plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

<p> From the visualization, we can guess the topics should be about Donald Trump, Uber, Bank, Tesla, minister. We will check it out in another notebook. Now we save the processed_docs oject for latter use</p>

In [0]:
processed_docs = processed_docs.values.tolist()

write_pickle_file(processed_docs, datapath + 'processed_docs.pkl')

<p> Create gensim dictionary  from processed_docs</p>

In [0]:
# filter out the less common words
# Keep tokens which are contained in at least 15 documents
# Keep tokens which are contained in no more than 50% documents
# Keep only the first 10000 most frequent tokens
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)
dictionary = corpora.Dictionary(processed_docs)

print("1st word in dictionary: {}".format(dictionary[0]))

print("1OOth word in dictionary: {}".format(dictionary[100]))

<p>Convert processed_docs into bad of word (bow) using dictionary </p>
<p> It is a list of word id and theirs frequencies </p>

In [0]:
bow = [dictionary.doc2bow(doc) for doc in processed_docs]

<p>View the first document in bow and using dictionary to see which words and their frequency</p>

In [0]:
doc = bow[1]

[(dictionary[id], freq) for id, freq in doc]


<p> Save bag-of_word </p>

In [0]:
write_pickle_file(bow, datapath+'bow.pkl')

"""
for id, freq in doc:
  print("{} : {}".format(dictionary[id], freq))
"""



<p>We build Tf-Idf, a simple transformation which takes documents represented as bag-of-words counts and applies a weighting which discounts common terms or equivalently, promotes rare terms </p>
<p> Then, save tfidf for using later</p>

In [0]:
tfidf_model = gensim.models.TfidfModel(bow)
tfidf = tfidf_model[bow]

write_pickle_file(tfidf, datapath + "tfidf.pkl")

In [0]:
doc = tfidf[1]
[(dictionary[id], freq) for id, freq in doc]

<p>Finally, save gensim dictionary object</p>

In [0]:
write_pickle_file(dictionary, datapath + 'dictionary.pkl')